In [1]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [2]:
import sys
sys.path.insert(0,"./../")

In [3]:
import os
import dotenv
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
from textblob import TextBlob
from pypdf import PdfReader
import nltk
from nltk.corpus import stopwords

In [4]:
dotenv.load_dotenv()

True

In [5]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nelsonlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
pdf_path = "../documents/GIS+Prospectus+UK.pdf"

In [7]:
pdf_reader = PdfReader(pdf_path)

In [8]:
pdf_page_parse_dict_list = []

In [9]:
# Split document to list of pages
for page_num, page in enumerate(pdf_reader.pages):
     text = page.extract_text()
     pdf_page_parse_dict = {}
     pdf_page_parse_dict['page_num'] = page_num
     pdf_page_parse_dict['text'] = text
     pdf_page_parse_dict_list.append(pdf_page_parse_dict)


In [24]:
# pdf_page_parse_dict_list[96]['text']

In [11]:
df_page  = pd.DataFrame(pdf_page_parse_dict_list)

In [12]:
# split pages to sentences
pdf_sentence_dict_list =[]
for page_dict in  tqdm(pdf_page_parse_dict_list):
     page_num = page_dict['page_num']
     text = page_dict['text']

     blob = TextBlob(text)
     for index,sentence in enumerate(blob.sentences):
          sentence_dict = {}
          sentence_dict['page_num'] = page_num
          sentence_dict['sentence_index'] = index
          sentence_dict['sentence'] = str(sentence)
          pdf_sentence_dict_list.append(sentence_dict)

  0%|          | 0/882 [00:00<?, ?it/s]

100%|██████████| 882/882 [00:00<00:00, 2990.45it/s]


In [13]:
df_sentence  = pd.DataFrame(pdf_sentence_dict_list)

### Extract Function Name

In [39]:
def extract_fund_name(text):
    import re
    
    text = text.replace("\n","")
    pattern = r'\b(?:[^a-zA-Z]*([A-Z][a-zA-Z ]+Fund)[^a-zA-Z]*)\b'
    matches = re.findall(pattern, text)

    filters = []
    for name in matches:
        if "  " in name:
            #  skip if more than two continuous space
            continue
        # all words should be uppercase at the first letter
        if name == " ".join([w.capitalize() for w  in name.split(" ")]):
            # split more funds if the extracted name contains more "Fund"
            filters.extend([n.lstrip() + "Fund" for n in name.split("Fund")])

    return filters

In [40]:
# text = pdf_page_parse_dict_list[96]['text']

In [44]:
# extract_fund_name(text)

In [45]:
df_sentence['fund_names'] = df_sentence['sentence'].apply(extract_fund_name)

In [46]:
all_names = []
for names in df_sentence[df_sentence['fund_names'].apply(lambda x: len(x)!=0)]['fund_names']:
    all_names.extend(names)


In [47]:
all_names = list(set(all_names))

In [48]:
all_names.sort(key = lambda x: len(x))

In [49]:
#  Merge short fund name to long fun name: 
#  For Example: "Market Bond Fund" should belong to "Global Market Bond Fund"  
merged_names = all_names.copy() 
for (index,name) in enumerate(all_names):
    for next_name in all_names[index+1:]:
        if name in next_name:
            all_names[index]=""
            continue

In [50]:
all_names =  [name for name in all_names if name]

In [51]:
#  Stop Fund Names;
#  For example: "This Fund", "A Fund","Mty Fund" should not be eligible
stop_fund_names = set(word.capitalize() + " Fund"  for word in stopwords.words('english'))

In [52]:
all_names =  [name for name in all_names if name not in stop_fund_names]

In [53]:
#  All qualified fund name
all_names

['Term Fund',
 'Asset Fund',
 'Selected Fund',
 'Original Fund',
 'Reporting Fund',
 'Euro Bond Fund',
 'Underlying Fund',
 'Euro Credit Fund',
 'Call Risk A Fund',
 'Global Bond Fund',
 'Dynamic Bond Fund',
 'State Street Fund',
 'Long Duration Fund',
 'Mortgage Risk A Fund',
 'Euro Income Bond Fund',
 'Arbitrage Risk A Fund',
 'Strategic Income Fund',
 'Global Advantage Fund',
 'Commodity Risk A Fund',
 'Irish Real Estate Fund',
 'Total Return Bond Fund',
 'Diversified Income Fund',
 'Term Opportunities Fund',
 'Currency Hedging A Fund',
 'Real Estate Risk A Fund',
 'Global Real Return Fund',
 'Emerging Local Bond Fund',
 'Term Local Currency Fund',
 'Low Duration Income Fund',
 'Low Average Duration Fund',
 'Factor Europe Equity Fund',
 'Energy Infrastructure Fund',
 'Emerging Markets Bond Fund',
 'Commodity Real Return Fund',
 'International Monetary Fund',
 'Global High Yield Bond Fund',
 'Mortgage Opportunities Fund',
 'National Pension Reserve Fund',
 'Euro Long Average Duration